In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
ls

In [ ]:
!pip install ipython-autotime

In [ ]:
%load_ext autotime

In [ ]:
!pip install -U opencv-python

## Parte 1: Cálculo de puntos de interés Harris

In [ ]:
%load_ext Cython

In [ ]:
%%cython
import cython
import numpy as np
cimport numpy as np

cpdef np.ndarray[np.float32_t, ndim=2] gradx(np.ndarray[np.float32_t, ndim=2] input):
  # POR HACER: calcular el gradiente en x
  cdef np.ndarray[np.float32_t, ndim=2] output=np.zeros([input.shape[0], input.shape[1]], dtype = np.float32)

  return output

In [ ]:
%%cython
import cython
import numpy as np
cimport numpy as np

cpdef np.ndarray[np.float32_t, ndim=2] grady(np.ndarray[np.float32_t, ndim=2] input):
  # POR HACER: Calcular el gradiente en y
  cdef np.ndarray[np.float32_t, ndim=2] output=np.zeros([input.shape[0], input.shape[1]], dtype = np.float32)

  return output

In [ ]:
%%cython
import cython
import numpy as np
cimport numpy as np

cpdef np.ndarray[np.float32_t, ndim=2] product(np.ndarray[np.float32_t, ndim=2] input1, np.ndarray[np.float32_t, ndim=2] input2):
  # POR HACER: generar una matriz que contenga el producto entre input1 e input2, pixel a pixel

  cdef np.ndarray[np.float32_t, ndim=2] output=np.zeros([input1.shape[0], input1.shape[1]], dtype = np.float32)
  return output

In [ ]:
%%cython
import cython
import numpy as np
cimport numpy as np

cpdef np.ndarray[np.float32_t, ndim=2] harris(np.ndarray[np.float32_t, ndim=2] mxx, np.ndarray[np.float32_t, ndim=2] mxy, np.ndarray[np.float32_t, ndim=2] myy):
  # POR HACER: Calcular el filtro de Harris a partir de (mxx, mxy, myy)
  cdef np.ndarray[np.float32_t, ndim=2] output=np.zeros([mxx.shape[0], mxx.shape[1]], dtype = np.float32)

  return output

In [ ]:
%%cython
import cython
import numpy as np
cimport numpy as np

cpdef np.ndarray[np.float32_t, ndim=2] getMaxima(np.ndarray[np.float32_t, ndim=2] h, float val):
  # POR HACER: Crear una imagen, inicialmente llena con ceros.
  # Luego, hacer 1 los valores en los cuales se cumplen dos condiciones:
  #   h[r,c] es un maximo local respecto a sus 8 vecinos
  #   h[r,c] supera el valor val
  cdef np.ndarray[np.float32_t, ndim=2] output=np.zeros([h.shape[0], h.shape[1]], dtype = np.float32)

  return output

In [ ]:
# Esta funcion genera keypoints a partir de una imagen de entrada
# La imagen de entrada es la salida del filtro de Harris
# No es necesario modificar esta funcion
def getKeyPoints(input):
  output = []
  input_rows = input.shape[0]
  input_cols = input.shape[1]
  for r in range(input_rows):
    for c in range(input_cols):
      if input[r,c] > 0:
        kp = cv2.KeyPoint()
        kp.pt = (c,r)
        kp.size = 10
        kp.angle = 0
        output.append(kp)
  return output

In [ ]:
def harrisDetector(input, val):
  input = np.float32( input )
  # Por hacer: calcular el filtro de Harris
  # Hay que realizar los siguientes pasos:
  # 1) Suavizar la imagen de entrada con cv.GaussianBlur( )
  # 2) Calcular gradientes imx e imy (usando funciones de cython gradx y grady definidas arriba)
  # 3) Calcular momentos usando la funcion product( )
  #    imxx = imx*imx (pixel a pixel)
  #    imxy = imx*imy (pixel a pixel)
  #    imyy = imy*imy (pixel a pixel)
  # 4) Suavizar momentos imxx, imxy, imyy con cv.GaussianBlur( )
  # 5) Aplicar el filtro de Harris (usando funcion de Cython harris definida arriba)
  # 6) Encontrar puntos maximos usando getMaxima( )
  # 7) Generar el listado de puntos usando getKeyPoints( )
  # 8) Devolver los puntos e interes y la imagen filtrada

  h = input    # Esta linea debe ser eliminada
  points = []  # Esta linea debe ser eliminada
  return points, np.float32(h)

In [ ]:
def do_rotate(img, angle):
  h = img.shape[0]
  w = img.shape[1]
  cx = w // 2
  cy = h // 2
  m = cv2.getRotationMatrix2D((cx, cy), -angle, 1.0)
  cosa = np.cos(angle * np.pi / 180.0)
  sina = np.sin(angle * np.pi / 180.0)
  nw = int((h * sina) + (w * cosa))
  nh = int((h * cosa) + (w * sina))
  m[0,2] += (nw / 2) - cx
  m[1,2] += (nh / 2) - cy
  return cv2.warpAffine(img, m, (nw, nh))

In [ ]:
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

img1 = cv2.imread('uch013a.jpg',0)
img2 = do_rotate(img1, 30)

kp1, h1 = harrisDetector(img1, 3e7)
kp2, h2 = harrisDetector(img2, 3e7)

res1 = cv2.drawKeypoints(img1, kp1, img1, None, cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
res2 = cv2.drawKeypoints(img2, kp2, img2, None, cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

print('Resultados de filtro de Harris')
cv2_imshow(h1 * 500 / np.max(np.max(h1)))
cv2_imshow(h2 * 500 / np.max(np.max(h2)))
print('Puntos de interes')
cv2_imshow(res1)
cv2_imshow(res2)


## Parte 2: Reconocimiento de objetos particulares

In [ ]:
import math

def genTransform(match, keypoints1, keypoints2):
  # Calcular transformacion de semejanza (e,theta,tx,ty) a partir de un calce "match".
  # Se debe notar que los keypoints de OpenCV tienen orientacion en grados.
  e = 1
  theta = 0
  tx = 0
  ty = 0
  return (e, theta, tx, ty)

In [ ]:
def computeConsensus(matches, keypoints1, keypoints2, e, theta, tx, ty, umbralpos):
  # Calcular el número de matches compatibles con la transformación (e, theta, tx, ty) indicada
  # Debe devolver el número de matches compatibles, y una lista con los matches compatibles
  return (0, matches)

In [ ]:
import random

def ransac(matches, keypoints1, keypoints2):
  accepted = []

  # Por hacer:
  # Implementar RANSAC. Se debe elegir un calce al azar y calcular su transformacion de semejanza (e,theta,tx,ty)
  #  usando la funcion genTransform( ) definida arriba
  # Luego se debe evaluar el consenso de la transformacion usando computeConsensus()
  # Esto se debe repetir varias veces
  # Una vez hechos todos los intentos, se debe analizar la hipotesis que tuvo el mayor consenso
  # Si dicho consenso es mayor a un umbral, se aceptan los calces del consenso como correctos
  # Los calces considerados correctos se deben guardar en "accepted"

  return accepted;

In [ ]:
def calcAfin(matches, keypoints1, keypoints2):
  # Por hacer: calcular la transformacion afin mediante minimos cuadrados a partir de "matches"
  return np.array([[1,0,0],[0,1,0]])

In [ ]:
# Esta función devuelve la imagen "input1" con un romboide dibujado
# El romboide representa el rectangulo de la imagen "input2" proyectada en la imagen "input1"
# La proyección se realiza a partir de la transformación "transf"

def drawProjAfin(transf, input1, input2):
  w = input2.shape[1];
  h = input2.shape[0];
  sq1 = [0,0]
  sq2 = [w-1,0]
  sq3 = [w-1,h-1]
  sq4 = [0,h-1];
  p1 = [0,0]
  p2 = [0,0]
  p3 = [0,0]
  p4 = [0,0]
  print("Tamano "  + str(transf.shape[1]) + " x " + str(transf.shape[0]));
  print("Posicion " + str(transf[0,2]) + " , " + str(transf[1,2]));
  print("Matriz " + str(transf[0,0]) + " " + str(transf[0,1]) + " " + str(transf[0,2]) + " " + str(transf[1,0]) + " " + str(transf[1,1]) + " " + str(transf[1,2]));

  p1[0] = transf[0,0] * sq1[0] + transf[0,1] * sq1[1] + transf[0,2];
  p1[1] = transf[1,0] * sq1[0] + transf[1,1] * sq1[1] + transf[1,2];
  p2[0] = transf[0,0] * sq2[0] + transf[0,1] * sq2[1] + transf[0,2];
  p2[1] = transf[1,0] * sq2[0] + transf[1,1] * sq2[1] + transf[1,2];
  p3[0] = transf[0,0] * sq3[0] + transf[0,1] * sq3[1] + transf[0,2];
  p3[1] = transf[1,0] * sq3[0] + transf[1,1] * sq3[1] + transf[1,2];
  p4[0] = transf[0,0] * sq4[0] + transf[0,1] * sq4[1] + transf[0,2];
  p4[1] = transf[1,0] * sq4[0] + transf[1,1] * sq4[1] + transf[1,2];

  p1 = (int(p1[0]), int(p1[1]))
  p2 = (int(p2[0]), int(p2[1]))
  p3 = (int(p3[0]), int(p3[1]))
  p4 = (int(p4[0]), int(p4[1]))


  out = np.copy(input1);
  cv2.line(out, p1, p2, (255,255,255));
  cv2.line(out, p2, p3, (255,255,255));
  cv2.line(out, p3, p4, (255,255,255));
  cv2.line(out, p4, p1, (255,255,255));
  return out;

In [ ]:
# Esta funcion ya esta lista, no debe ser modificada
def filterMatches(matches):
  # Apply ratio test
  points1 = []
  points2 = []
  good = []

  for m,n in matches:
    if m.distance < 0.75*n.distance:  # 0.75
      good.append(m)
      points1.append(kp1[m.queryIdx].pt)
      points2.append(kp2[m.trainIdx].pt)
  return np.array(points1), np.array(points2), good

In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
from google.colab.patches import cv2_imshow

img2 = cv2.imread('uch013a.jpg',0)
img1 = cv2.imread('uch013b.jpg',0)

# Initiate SIFT detector
sift = cv2.SIFT_create()

# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(img1,None)
kp2, des2 = sift.detectAndCompute(img2,None)

#### BFMatcher with default params
bf = cv2.BFMatcher()
matches = bf.knnMatch(des1,des2, k=2)
points1, points2, good = filterMatches(matches)

img_match = cv2.drawMatches(img1,kp1,img2,kp2,good, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

cv2_imshow(img_match)


In [ ]:
# Por hacer:
# 1) Obtener los calces aceptados a partir de la función ransac(good, kp2, kp1)
# 2) Dibujar los calces aceptados
# 3) Calcular la transformacion afin
# 4) Dibujar la imagen con el romboide superpuesto